# First steps with `SimPy`

In [25]:
import simpy

Basic concepts [(doc)](https://simpy.readthedocs.io/en/latest/simpy_intro/basic_concepts.html)

In [26]:
def car(env):
    while True:
        print(f"Start parking at {env.now}")
        parking_duration = 5
        yield env.timeout(parking_duration)

        print(f"Finished parking at {env.now}")
        driving_duration = 3
        yield env.timeout(driving_duration)

In [27]:
env = simpy.Environment()
process = env.process(car(env))
env.run(until=15)

Start parking at 0
Finished parking at 5
Start parking at 8
Finished parking at 13


Process Interaction [(doc)](https://simpy.readthedocs.io/en/latest/simpy_intro/process_interaction.html)

In [9]:
class Car(object):
    def __init__(self, env):
        self.env = env
        # Start the run process everytime an instance is created.
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We yield the process that process() returns
            # to wait for it to finish
            yield self.env.process(self.charge(charge_duration))

            # The charge process has finished and
            # we can start driving again.
            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [28]:
env = simpy.Environment()
car = Car(env)
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


Interupting

In [12]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        yield self.env.timeout(duration)

In [13]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env, car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


Share Resources [(doc)](https://simpy.readthedocs.io/en/latest/simpy_intro/shared_resources.html)

In [21]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print(f'{name} arriving at {env.now}')
    with bcs.request() as req:
        yield req

        # Charge the battery
        print(f'{name} starting to charge at {env.now}')
        yield env.timeout(charge_duration)
        print(f'{name} leaving the bcs at {env.now}')

In [22]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)

In [23]:
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))


env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


In [70]:
def my_proc(n):
    for i in range(n):
        yield i
    return "DONE !!!!!!!!!!!!!!!"

g = my_proc(3)

In [74]:
try:
    print(next(g))
except StopIteration as e:
    print(dir(e))
    for a in dir(e):
        if not a.startswith('__'):
            print(f"{a}: {getattr(e, a)}")
    display(e)

['__cause__', '__class__', '__context__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__suppress_context__', '__traceback__', 'args', 'value', 'with_traceback']
args: ('DONE !!!!!!!!!!!!!!!',)
value: DONE !!!!!!!!!!!!!!!
with_traceback: <built-in method with_traceback of StopIteration object at 0x7f9e436a28c0>


StopIteration('DONE !!!!!!!!!!!!!!!')